# Text output inlezen

In [4]:
import numpy as np

def load_images(file_path):
    images = {}
    with open(file_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line.startswith('#'):
                continue
            elements = line.split()
            image_id = int(elements[0])
            qw, qx, qy, qz = map(float, elements[1:5])
            tx, ty, tz = map(float, elements[5:8])
            camera_id = int(elements[8])
            image_name = elements[9]

            line = f.readline()
            points2d = []
            elements = line.split()
            for i in range(0, len(elements), 3):
                x, y, point3d_id = map(float, elements[i:i+3])
                points2d.append([x, y, int(point3d_id)])
            points2d = np.array(points2d)

            images[image_name] = {
                'image_id': image_id,
                'qw': qw, 'qx': qx, 'qy': qy, 'qz': qz,
                'tx': tx, 'ty': ty, 'tz': tz,
                'camera_id': camera_id, 'image_name': image_name, 'points2d': points2d
            }
    return images

def load_points3d(file_path):
    points3d = {}
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            elements = line.split()
            point_id = int(elements[0])
            xyz = np.array(elements[1:4], dtype=float)
            rgb = np.array(elements[4:7], dtype=int)
            error = float(elements[7])
            track = np.array(elements[8:], dtype=int).reshape(-1, 2)
            points3d[point_id] = {'xyz': xyz, 'rgb': rgb, 'error': error, 'track': track}
    return points3d

def find_corresponding_3d_points(image_data, keypoints_2d, points3d):
    corresponding_3d_points = []
    for kp in keypoints_2d:
        x, y = kp
        distances = np.linalg.norm(image_data['points2d'][:, :2] - np.array([x, y]), axis=1)
        closest_point_idx = np.argmin(distances)
        point2d_id = image_data['points2d'][closest_point_idx, 2]
        if point2d_id != -1:
            corresponding_3d_points.append(points3d[point2d_id]['xyz'])
        else:
            corresponding_3d_points.append(None)
    return corresponding_3d_points

def interpolate_missing_point(p1, p2):
    if p1 is None and p2 is not None:
        return p2
    if p1 is not None and p2 is None:
        return p1
    if p1 is not None and p2 is not None:
        return (np.array(p1) + np.array(p2)) / 2
    return None

def ensure_all_points_mapped(corresponding_3d_points):
    n = len(corresponding_3d_points)
    for i in range(n):
        if corresponding_3d_points[i] is None:
            left = None
            right = None
            if i > 0:
                left = corresponding_3d_points[i-1]
            if i < n-1:
                right = corresponding_3d_points[i+1]
            corresponding_3d_points[i] = interpolate_missing_point(left, right)
    return corresponding_3d_points

def calculate_distance_3d(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))



images = load_images('sparse_text/images.txt')
points3d = load_points3d('sparse_text/points3D.txt')

In [5]:
images

{'eastbound/eastbound_20240319_07826.png': {'image_id': 1,
  'qw': 0.9885664846997114,
  'qx': -0.13166405620748858,
  'qy': -0.07050102177867344,
  'qz': -0.020748194128879315,
  'tx': -0.04210452406741276,
  'ty': -0.3102849047561297,
  'tz': 5.986669643013593,
  'camera_id': 1,
  'image_name': 'eastbound/eastbound_20240319_07826.png',
  'points2d': array([[ 3.74924591e+02,  7.99250122e+02, -1.00000000e+00],
         [ 4.58741058e+02,  3.06999451e+02, -1.00000000e+00],
         [ 2.44384399e+03,  5.12324097e+02, -1.00000000e+00],
         ...,
         [ 2.45068335e+03,  4.00462036e+02, -1.00000000e+00],
         [ 2.46934058e+03,  4.00534760e+02, -1.00000000e+00],
         [ 1.84377637e+03,  6.64885254e+02,  1.42500000e+03]])},
 'eastbound/eastbound_20240319_07846.png': {'image_id': 2,
  'qw': 0.9887205326475381,
  'qx': -0.13261891021262018,
  'qy': -0.06618873621539927,
  'qz': -0.02151706702582321,
  'tx': 0.00474802101350259,
  'ty': -0.2869067493496241,
  'tz': 5.68533521970799

In [1]:
prefix = 'eastbound/'
image_name = 'eastbound_20240319_07851.png'
images[prefix+image_name]

NameError: name 'images' is not defined

In [7]:
import cv2

image_path = 'eastbound_20240319_07851.png'


# Load the image
image = cv2.imread(image_path)

if image is None:
    print(f"Error loading image: {image_path}")
else:
    # Get the points2d for the specified image
    points2d = images['eastbound/eastbound_20240319_07851.png']['points2d']

    # Draw the points on the image
    for point in points2d:
        x, y, id = point
        if id != -1:
            cv2.circle(image, (int(x), int(y)), radius=3, color=(0, 255, 0), thickness=-1)

    # Display the image with the drawn points
    cv2.imshow('Image with 2D Points', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [87]:
images

{'eastbound/eastbound_20240319_07826.png': {'image_id': 1,
  'qw': 0.9885664846997114,
  'qx': -0.13166405620748858,
  'qy': -0.07050102177867344,
  'qz': -0.020748194128879315,
  'tx': -0.009734324709952426,
  'ty': 0.044854165071127085,
  'tz': -0.06139172651101565,
  'camera_id': 1,
  'image_name': 'eastbound/eastbound_20240319_07826.png',
  'points2d': array([[ 3.74924591e+02,  7.99250122e+02, -1.00000000e+00],
         [ 4.58741058e+02,  3.06999451e+02, -1.00000000e+00],
         [ 2.44384399e+03,  5.12324097e+02, -1.00000000e+00],
         ...,
         [ 2.45068335e+03,  4.00462036e+02, -1.00000000e+00],
         [ 2.46934058e+03,  4.00534760e+02, -1.00000000e+00],
         [ 1.84377637e+03,  6.64885254e+02,  1.42500000e+03]])},
 'eastbound/eastbound_20240319_07846.png': {'image_id': 2,
  'qw': 0.9887205326475381,
  'qx': -0.13261891021262018,
  'qy': -0.06618873621539927,
  'qz': -0.02151706702582321,
  'tx': -0.049286924640339985,
  'ty': 0.21906807532008385,
  'tz': -0.309644

In [8]:
points3d

{3786: {'xyz': array([-1.89439065,  0.61685054, -3.23238287]),
  'rgb': array([195, 200, 205]),
  'error': 1.2420440241392063,
  'track': array([[  11, 6740],
         [   4, 5510],
         [  13, 6946]])},
 69042: {'xyz': array([-1.12415572,  1.41734606, -4.64401963]),
  'rgb': array([115,  98,  72]),
  'error': 0.2479632182391044,
  'track': array([[  16, 4346],
         [   3, 8451]])},
 45694: {'xyz': array([-0.51804922, -0.23905279, -0.84408042]),
  'rgb': array([241, 240, 244]),
  'error': 0.508458123102649,
  'track': array([[   49, 12311],
         [   51, 11932],
         [   55, 11210]])},
 387: {'xyz': array([-0.49124167,  1.57183414, -4.47420504]),
  'rgb': array([42, 59, 24]),
  'error': 1.663871491510264,
  'track': array([[  11, 1021],
         [   6,  550],
         [   7, 1128],
         [  12,  371],
         [   4,  299],
         [  16, 2794],
         [   2, 3040],
         [   3, 5764]])},
 70251: {'xyz': array([-1.12924588,  0.81229568, -2.91249106]),
  'rgb': a

# gefilterde point cloud visualiseren - klad

In [9]:
import open3d as o3d
import numpy as np

# ply_path = 'exported_model.ply'
# pcd = o3d.io.read_point_cloud(ply_path)

# # Convert to numpy array for easier filtering
# points = np.asarray(pcd.points)
# filtered_points = points

points = np.array([point['xyz'] for point in points3d.values()])

# # Example: Fetch points within a certain range
x_min, x_max = -5.0, 5.0
y_min, y_max = -5.0, 5.0
z_min, z_max = -5.0, 5.0

# x_min, x_max = -1.0, 1.0
# y_min, y_max = -1.0, 1.0
# z_min, z_max = -1.0, 1.0

filtered_points = points[
    (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &
    (points[:, 1] >= y_min) & (points[:, 1] <= y_max) 
    # (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
]

print(len(filtered_points))


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
40506


In [12]:
# hier kan je point cloud met oorsprong visualiseren - KLAD

# Create a point cloud object for the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Create coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# Visualize the filtered points along with the axes
o3d.visualization.draw_geometries([filtered_pcd, axes])

# Hieronder klad - niet lezen hier pruts ik morgen mee verder

In [75]:
# Load data
# images = load_images('sparse/0/images.txt')
# points3d = load_points3d('sparse/0/points3D.txt')

# Extract the first camera position
first_image_key = list(images.keys())[8]
first_image = images[first_image_key]
first_camera_position = np.array([first_image['tx'], first_image['ty'], first_image['tz']])

def translate_model(images, points3d, translation):
    # Apply translation to all camera positions
    for image in images.values():
        image['tx'] -= translation[0]
        image['ty'] -= translation[1]
        image['tz'] -= translation[2]

    # Apply translation to all 3D points
    for point in points3d.values():
        point['xyz'] -= translation

    return images, points3d

# Translate the model to center around the first camera position
images, points3d = translate_model(images, points3d, first_camera_position)


first_image_key = list(images.keys())[8]
first_image = images[first_image_key]
first_camera_position = np.array([first_image['tx'], first_image['ty'], first_image['tz']])
first_camera_position

In [60]:
first_camera_position

array([0., 0., 0.])

In [81]:
first_image

{'image_id': 9,
 'qw': 0.9885343076284088,
 'qx': -0.13134495102014662,
 'qy': -0.07140580475744576,
 'qz': -0.021204658216520473,
 'tx': 0.0,
 'ty': 0.0,
 'tz': 0.0,
 'camera_id': 1,
 'image_name': 'eastbound/eastbound_20240319_07821.png',
 'points2d': array([[ 3.94870056e+02,  7.74810364e+02, -1.00000000e+00],
        [ 4.91178314e+02,  2.98714569e+02, -1.00000000e+00],
        [ 2.38948511e+03,  5.20148560e+02,  7.35410000e+04],
        ...,
        [ 2.64070898e+03,  3.58594025e+02, -1.00000000e+00],
        [ 9.28838654e+01,  3.59742554e+02, -1.00000000e+00],
        [ 1.45507324e+03,  6.07396362e+02,  3.11890000e+04]])}

In [92]:
def translate_model(points3d, translation):
    for point in points3d.values():
        point['xyz'] -= translation

def rotate_model(points3d, rotation):
    for point in points3d.values():
        point['xyz'] = rotation.apply(point['xyz'])

# Load data
images = load_images('sparse_text/images.txt')
points3d = load_points3d('sparse_text/points3D.txt')

# Extract the first camera position and rotation
first_image_key = list(images.keys())[8]
first_image = images[first_image_key]
first_camera_position = np.array([first_image['tx'], first_image['ty'], first_image['tz']])
first_camera_rotation = R.from_quat([first_image['qx'], first_image['qy'], first_image['qz'], first_image['qw']])

# Print the first camera position and rotation before translation
print("First camera position before translation:", first_camera_position)
print("First camera rotation before translation (quaternion):", [first_image['qx'], first_image['qy'], first_image['qz'], first_image['qw']])

# Translate the model to center around the first camera position
translate_model(points3d, first_camera_position)

# Print the first camera position after translation
print("First camera position after translation (should be close to zero):", first_camera_position - first_camera_position)

# Compute the inverse rotation to align the camera's axes with the global axes
inverse_rotation = first_camera_rotation.inv()

# Rotate the model to align the camera axes
rotate_model(points3d, inverse_rotation)

# Collect all points3d into a numpy array
points = np.array([point['xyz'] for point in points3d.values()])

# Define the coordinate ranges
x_min, x_max = -5.0, 5.0
y_min, y_max = -5.0, 5.0
z_min, z_max = -5.0, 5.0

# Filter the points based on the specified ranges
filtered_points = points[
    (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &
    (points[:, 1] >= y_min) & (points[:, 1] <= y_max) &
    (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
]

print(f"Number of filtered points: {len(filtered_points)}")

# Create a point cloud object for the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Create coordinate axes at the first camera position
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# Visualize the filtered points along with the axes
o3d.visualization.draw_geometries([filtered_pcd, axes])

First camera position before translation: [-0.05400498 -0.31758882  6.06203959]
First camera rotation before translation (quaternion): [-0.13134495102014662, -0.07140580475744576, -0.021204658216520473, 0.9885343076284088]
First camera position after translation (should be close to zero): [0. 0. 0.]
Number of filtered points: 58135


In [82]:

from scipy.spatial.transform import Rotation as R
first_camera_rotation = R.from_quat([first_image['qx'], first_image['qy'], first_image['qz'], first_image['qw']])

In [ ]:

def rotate_model(images, points3d, rotation):
    # Apply rotation to all camera positions
    for image in images.values():
        position = np.array([image['tx'], image['ty'], image['tz']])
        rotated_position = rotation.apply(position)
        image['tx'], image['ty'], image['tz'] = rotated_position

    # Apply rotation to all 3D points
    for point in points3d.values():
        point['xyz'] = rotation.apply(point['xyz'])

In [ ]:

# Compute the inverse rotation to align the camera's axes with the global axes
inverse_rotation = first_camera_rotation.inv()

# Rotate the model to align the camera axes
rotate_model(images, points3d, inverse_rotation)

# Collect all points3d into a numpy array
points = np.array([point['xyz'] for point in points3d.values()])

In [94]:
def apply_translation(points3d, translation):
    for point in points3d.values():
        point['xyz'] -= translation

def apply_rotation(points3d, rotation):
    for point in points3d.values():
        point['xyz'] = rotation.apply(point['xyz'])

def debug_print_camera_and_points(images, points3d, message):
    print(message)
    for image_name, image_data in images.items():
        print(f"Camera {image_name}: position = ({image_data['tx']:.2f}, {image_data['ty']:.2f}, {image_data['tz']:.2f})")
        break  # Print only the first camera for brevity

    sample_points = list(points3d.values())[:5]
    for i, point in enumerate(sample_points):
        print(f"Point {i}: ({point['xyz'][0]:.2f}, {point['xyz'][1]:.2f}, {point['xyz'][2]:.2f})")

# Load data
images = load_images('sparse_text/images.txt')
points3d = load_points3d('sparse_text/points3D.txt')

# Extract the first camera position and rotation (world-to-camera)
first_image_key = list(images.keys())[0]
first_image = images[first_image_key]
camera_to_world_rotation = R.from_quat([first_image['qx'], first_image['qy'], first_image['qz'], first_image['qw']]).inv()
camera_to_world_translation = -camera_to_world_rotation.apply([first_image['tx'], first_image['ty'], first_image['tz']])

# Print the first camera position and rotation before translation
debug_print_camera_and_points(images, points3d, "Before translation and rotation:")

# Translate the model to center around the first camera position
apply_translation(points3d, camera_to_world_translation)
apply_translation(images, camera_to_world_translation)

# Print the first camera position after translation
debug_print_camera_and_points(images, points3d, "After translation:")

# Rotate the model to align the camera axes
apply_rotation(points3d, camera_to_world_rotation)
apply_rotation(images, camera_to_world_rotation)

# Print the camera position and points after rotation
debug_print_camera_and_points(images, points3d, "After rotation:")

# Collect all points3d into a numpy array
points = np.array([point['xyz'] for point in points3d.values()])

# Define the coordinate ranges
x_min, x_max = -5.0, 5.0
y_min, y_max = -5.0, 5.0
z_min, z_max = -5.0, 5.0

# Filter the points based on the specified ranges
filtered_points = points[
    (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &
    (points[:, 1] >= y_min) & (points[:, 1] <= y_max) &
    (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
]

print(f"Number of filtered points: {len(filtered_points)}")

# Create a point cloud object for the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Create coordinate axes at the origin
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# Visualize the filtered points along with the axes
o3d.visualization.draw_geometries([filtered_pcd, axes])

Before translation and rotation:
Camera eastbound/eastbound_20240319_07826.png: position = (-0.04, -0.31, 5.99)
Point 0: (-1.89, 0.62, -3.23)
Point 1: (-1.12, 1.42, -4.64)
Point 2: (-0.52, -0.24, -0.84)
Point 3: (-0.49, 1.57, -4.47)
Point 4: (-1.13, 0.81, -2.91)


KeyError: 'xyz'

In [ ]:
import sqlite3

# Connect to the database
database_path = 'database.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Fetch the first 30 rows from a table
cursor.execute("SELECT * FROM keypoints LIMIT 30")
rows = cursor.fetchall()

# Print the fetched rows
for row in rows:
    print(row)

# Close the database connection
conn.close()

In [3]:
import sqlite3

# Connect to the COLMAP database
database_path = 'database.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Get a list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# Print the schema of each table
for table_name in tables:
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    print(f"Schema of table {table_name[0]}:")
    print(cursor.fetchall())

# Close the database connection
conn.close()


Tables in the database: [('cameras',), ('sqlite_sequence',), ('images',), ('keypoints',), ('descriptors',), ('matches',), ('two_view_geometries',)]
Schema of table cameras:
[(0, 'camera_id', 'INTEGER', 1, None, 1), (1, 'model', 'INTEGER', 1, None, 0), (2, 'width', 'INTEGER', 1, None, 0), (3, 'height', 'INTEGER', 1, None, 0), (4, 'params', 'BLOB', 0, None, 0), (5, 'prior_focal_length', 'INTEGER', 1, None, 0)]
Schema of table sqlite_sequence:
[(0, 'name', '', 0, None, 0), (1, 'seq', '', 0, None, 0)]
Schema of table images:
[(0, 'image_id', 'INTEGER', 1, None, 1), (1, 'name', 'TEXT', 1, None, 0), (2, 'camera_id', 'INTEGER', 1, None, 0), (3, 'prior_qw', 'REAL', 0, None, 0), (4, 'prior_qx', 'REAL', 0, None, 0), (5, 'prior_qy', 'REAL', 0, None, 0), (6, 'prior_qz', 'REAL', 0, None, 0), (7, 'prior_tx', 'REAL', 0, None, 0), (8, 'prior_ty', 'REAL', 0, None, 0), (9, 'prior_tz', 'REAL', 0, None, 0)]
Schema of table keypoints:
[(0, 'image_id', 'INTEGER', 1, None, 1), (1, 'rows', 'INTEGER', 1, None,

In [23]:
import open3d as o3d
import numpy as np

ply_path = 'exported_model.ply'
pcd = o3d.io.read_point_cloud(ply_path)

# Convert to numpy array for easier filtering
points = np.asarray(pcd.points)
# filtered_points = points

# # Example: Fetch points within a certain range
x_min, x_max = -5.0, 5.0
y_min, y_max = -5.0, 5.0
z_min, z_max = -5.0, 5.0

filtered_points = points[
    (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &
    (points[:, 1] >= y_min) & (points[:, 1] <= y_max) 
    # (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
]

print(len(filtered_points))


109168


In [24]:
# Create a point cloud object for the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Visualize the filtered points
o3d.visualization.draw_geometries([filtered_pcd])

In [6]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Connect to the COLMAP database
database_path = 'database.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Read points from points3D table
cursor.execute("SELECT * FROM keypoints")
points3D = cursor.fetchall()

# Close the database connection
conn.close()

# Extract coordinates
points = np.array([[point[1], point[2], point[3]] for point in points3D])

print(points)

# Scatter plot using matplotlib
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='b', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('3D Points Scatter Plot')
plt.show()

# Scatter plot using plotly
fig = px.scatter_3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], 
                    title='3D Points Scatter Plot', labels={'x':'X', 'y':'Y', 'z':'Z'})
fig.show()


OperationalError: no such function: head